In [34]:
!pip install gradio jinja2

In [35]:
import gradio as gr

In [36]:
import tensorflow as tf
model = tf.keras.models.load_model('toxicity.h5')

In [37]:
# to use the same vectorizer we created on training we are recreating it here - see if more optimum method exists to access the vectorizer form training file like pickling it,etc?
import pandas as pd
df = pd.read_csv(r'C:\Users\Atul\Downloads\data_toxicity_comments\data.csv')  #Since training data is large enough we use this as our entire dataset
from tensorflow.keras.layers import TextVectorization
x = df['comment_text']
y = df.iloc[:,2:].values
max_features = 200000 #number of words we want to include in our dictionary
vectorizer = TextVectorization(max_tokens = max_features, 
                                output_sequence_length = 1800,  #each sentence in our input we will cap at 1800 words and wont take words beyond that so that each sentence's mapped array is of length 1800
                                  output_mode = 'int') # want our encodings to be integers
vectorizer.adapt(x.values)  #making our vectorizer learn the dictionary by passing all our comments represneted as a numpy array


C:\Users\Atul\AppData\Local\Temp\ipykernel_4484\1990041617.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df_toxic_sample.append(df_non_toxic_sample)


In [38]:
def score_comment(comment):
    vectorized_comment = vectorizer([comment])
    results = model.predict(vectorized_comment)
    
    text = ''
    for idx,col in enumerate(df.columns[2:]):
        text += '{}: {}\n'.format(col, results[0][idx]>0.5)
        
    return text

In [39]:
interface = gr.Interface(fn = score_comment, inputs = gr.inputs.Textbox(lines=2, placeholder = 'Comment to Score'),
                        outputs = 'text')

C:\Users\Atul\anaconda3\lib\site-packages\gradio\deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
C:\Users\Atul\anaconda3\lib\site-packages\gradio\deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)
C:\Users\Atul\anaconda3\lib\site-packages\gradio\deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)


In [40]:
interface.launch(share = True)

Running on local URL:  http://127.0.0.1:7865/
Running on public URL: https://56029.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<gradio.routes.App at 0x1e09ecddca0>,
 'http://127.0.0.1:7865/',
 'https://56029.gradio.app')

1/1 [==============================] - 0s 81ms/step
